### This notebook aims to generate smaller dataset we need for further use. With this memory recycle mecanism, we merely need less than 2G memory. Hash table method is used for speed reason.

In [6]:
import xml.etree.ElementTree as et 
from datetime import datetime, date
import csv
import numpy as np
import pandas as pd
import gc
from collections import defaultdict

In [2]:
with open("./Data/Posts.xml","r") as f: 
    for i in range(10):
        print (f.readline())

﻿<?xml version="1.0" encoding="utf-8"?>

<posts>

  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" 

In [3]:
#Get Posts header table
counter = 0
filePath = './Data/Posts.xml'
parser = et.iterparse(filePath)
headers = []
for event,elem in parser:
    for key in elem.attrib.keys(): 
        if key not in headers:
            headers.append(key)
    counter += 1
    if counter>10:
        break
    elem.clear()
print (headers)
headers.remove('LastEditDate')
headers.remove('LastActivityDate')
headers.remove('CommunityOwnedDate')

['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId', 'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount', 'CommunityOwnedDate', 'ParentId', 'OwnerDisplayName', 'ClosedDate']


In [4]:
parser = et.iterparse(filePath) #redifine is necessary
try:
    with open('./Question_train.csv', mode='w') as train, open('./Question_test.csv', mode='w') as test:
        train_writer = csv.DictWriter(train, delimiter=',', fieldnames=headers)
        train_writer.writeheader()
    
        test_writer = csv.DictWriter(test, delimiter=',', fieldnames=headers)
        test_writer.writeheader()

        for event,elem in parser:
            row = {}
            if elem.get('CreationDate') != None and elem.get('PostTypeId') == '1':
                date_string = elem.get('CreationDate')  
            else:
                elem.clear()
                continue
            date_object = datetime.strptime(date_string[:-4], '%Y-%m-%dT%H:%M:%S')
        
            if date_object.date()<=date(2013, 3, 31) and date_object.date()>=date(2013, 1, 1):
                for header in headers: 
                    row[header] = elem.get(header)
                train_writer.writerow(row)
            elif date_object.date()<=date(2013, 4, 30) and date_object.date()>=date(2013, 4, 1):
                for header in headers:
                    row[header] = elem.get(header)
                test_writer.writerow(row)
            elem.clear()
except IOError as e:
    print ('Operation failed: %s' % e.strerror)


In [16]:
train_df = pd.read_csv('./Question_train.csv')
train_checktable = train_df['Id'].tolist()
dic_train = defaultdict(int) 
for item in train_checktable:
    dic_train[item] = 1 

test_df = pd.read_csv('./Question_test.csv')
test_checktable = test_df['Id'].tolist()
dic_test = defaultdict(int)
for item in test_checktable:
    dic_test[item] = 1 
    

filePath = './Data/Posts.xml'
parser = et.iterparse(filePath) #redifine is necessary
try:
    with open('./Answer_train.csv', mode='w') as train, open('./Answer_test.csv', mode='w') as test:
        train_writer = csv.DictWriter(train, delimiter=',', fieldnames=headers)
        train_writer.writeheader()
    
        test_writer = csv.DictWriter(test, delimiter=',', fieldnames=headers)
        test_writer.writeheader()

        for event,elem in parser:
            row = {}
            if elem.get('ParentId') != None and int(elem.get('ParentId')) in dic_train and elem.get('PostTypeId') == '2':
                for header in headers: 
                    row[header] = elem.get(header)
                train_writer.writerow(row)
            elif elem.get('ParentId') != None and int(elem.get('ParentId')) in dic_test and elem.get('PostTypeId') == '2':
                for header in headers:
                    row[header] = elem.get(header)
                test_writer.writerow(row)
            elem.clear()
except IOError as e:
    print ('Operation failed: %s' % e.strerror)
    

In [6]:
with open("./Data/Comments.xml","r") as f: 
    for i in range(10):
        print (f.readline())

﻿<?xml version="1.0" encoding="utf-8"?>

<comments>

  <row Id="2" PostId="35314" Score="9" Text="Yeah, I didn't believe it until I created a console app - but good lord!  Why would they give you the rope to hang yourself!  I hated that about VB.NET - the OrElse and AndAlso keywords!" CreationDate="2008-09-06T08:09:52.330" UserId="3" />

  <row Id="4" PostId="35195" Score="0" Text="I don't see an accepted answer now, I wonder how that got unaccepted. Incidentally, I would have marked an accepted answer based on the answers available at the time. Also, accepted doesn't mean Best :)" CreationDate="2008-09-06T08:42:16.980" UserId="380" />

  <row Id="10" PostId="45651" Score="6" Text="It will help if you give some details of which database you are using as techniques vary." CreationDate="2008-09-06T13:38:23.647" UserId="242" />

  <row Id="12" PostId="47428" Score="3" Text="One of the things that make a url user-friendly is 'discover-ability', meaning you can take a guess at url's simply 

In [18]:
#Get Comments header table
counter = 0
filePath = './Data/Comments.xml'
parser = et.iterparse(filePath)
headers = []
for event,elem in parser:
    for key in elem.attrib.keys(): 
        if key not in headers:
            headers.append(key)
    counter += 1
    if counter>10:
        break
    elem.clear()
print (headers)

['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId']


In [20]:
train_df2 = pd.read_csv('./Answer_train.csv')
Atrain_checktable = train_df2['Id'].tolist()

test_df2 = pd.read_csv('./Answer_test.csv')
Atest_checktable = test_df2['Id'].tolist()

train_checklist = [*train_checktable, *Atrain_checktable]
dic_train = defaultdict(int) 
for item in train_checklist:
    dic_train[item] = 1 

test_checklist = [*test_checktable, *Atest_checktable]
dic_test = defaultdict(int)
for item in test_checklist:
    dic_test[item] = 1 


del test_df, train_df, test_df2, train_df2
gc.collect() 

parser = et.iterparse(filePath) #redifine is necessary
try:
    with open('./Comments_train.csv', mode='w') as train, open('./Comments_test.csv', mode='w') as test:
        train_writer = csv.DictWriter(train, delimiter=',', fieldnames=headers)
        train_writer.writeheader()
    
        test_writer = csv.DictWriter(test, delimiter=',', fieldnames=headers)
        test_writer.writeheader()

        for event,elem in parser:
            row = {}
            if elem.get('PostId') != None and int(elem.get('PostId')) in dic_train:
                for header in headers: 
                    row[header] = elem.get(header)
                train_writer.writerow(row)
            elif elem.get('ParentId') != None and int(elem.get('PostId')) in dic_test:
                for header in headers:
                    row[header] = elem.get(header)
                test_writer.writerow(row)
            elem.clear()
except IOError as e:
    print ('Operation failed: %s' % e.strerror)

In [23]:
with open("./Data/Users.xml","r") as f: 
    for i in range(10):
        print (f.readline())

﻿<?xml version="1.0" encoding="utf-8"?>

<users>

  <row Id="-1" Reputation="1" CreationDate="2008-07-31T00:00:00.000" DisplayName="Community" LastAccessDate="2008-08-26T00:16:53.810" WebsiteUrl="http://meta.stackexchange.com" Location="on the server farm" AboutMe="&lt;p&gt;Hi, I'm not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I'm a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;  &lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;  &lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;  &lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;  &lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;  &lt;li&gt;&lt;a href=&quot;https://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" Views="649" 

In [28]:
#Get User header table
counter = 0
filePath = './Data/Users.xml'
parser = et.iterparse(filePath)
headers = []
for event,elem in parser:
    for key in elem.attrib.keys(): 
        if key not in headers:
            headers.append(key)
    counter += 1
    if counter>10:
        break
    elem.clear()
print (headers)
headers.remove('WebsiteUrl')
headers.remove('ProfileImageUrl')


['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes', 'AccountId', 'ProfileImageUrl']


In [29]:
train_df1 = pd.read_csv('./Question_train.csv')
test_df1 = pd.read_csv('./Question_test.csv')

train_df2 = pd.read_csv('./Answer_train.csv')
test_df2 = pd.read_csv('./Answer_test.csv')

train_df3 = pd.read_csv('./Comments_train.csv')
test_df3 = pd.read_csv('./Comments_test.csv')

from collections import OrderedDict #Compatible for both python 2 and python 3

usersTrain_check = OrderedDict.fromkeys([*train_df1['OwnerUserId'].tolist(),*train_df2['OwnerUserId'].tolist(),*train_df3['UserId'].tolist()])
usersTest_check = OrderedDict.fromkeys([*test_df1['OwnerUserId'].tolist(),*test_df2['OwnerUserId'].tolist(),*test_df3['UserId'].tolist()])

del test_df1, train_df1, test_df2, train_df2, test_df3, train_df3
gc.collect()                    
                       
parser = et.iterparse(filePath) #redifine is necessary
try:
    with open('./Users_train.csv', mode='w') as train, open('./Users_test.csv', mode='w') as test:
        train_writer = csv.DictWriter(train, delimiter=',', fieldnames=headers)
        train_writer.writeheader()
    
        test_writer = csv.DictWriter(test, delimiter=',', fieldnames=headers)
        test_writer.writeheader()

        for event,elem in parser:
            row = {}
            if elem.get('Id') != None and int(elem.get('Id')) in usersTrain_check:
                for header in headers: 
                    row[header] = elem.get(header)
                train_writer.writerow(row)
            elif elem.get('Id') != None and int(elem.get('Id')) in usersTest_check:
                for header in headers:
                    row[header] = elem.get(header)
                test_writer.writerow(row)
            elem.clear()
except IOError as e:
    print ('Operation failed: %s' % e.strerror)                       